In [ ]:
import gc
import time
import argparse
from fastai.layers import swish

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

from fastai.callback.schedule import Learner
from fastai.data.core import DataLoaders
from fastai.losses import L1LossFlat
from fastai.callback.core import Callback
from fastai.callback.tracker import ReduceLROnPlateau, SaveModelCallback

from HW_base import evaluate_build, focus_build
from HW_torch import dataLoads_build, net_parameter_count, hw_layer
from HW_torch import torch_valid, torch_predict

In [ ]:
fname               = '(F5-E128-F10)_RES(LSTM-FC-HW)x8-(FC-SELU-FC)'
evaluate_num        = 128
focus_min           = 0.1
net_block_num       = 8
hw_active           = False
epoch_num_first     = 100
batch_size_first    = 100
epoch_num_second    = 200
batch_size_second   = 800

In [ ]:
class Net_block(torch.nn.Module):
    def __init__(self, input_dims, internal_dims, output_dims, bias=True, bidirectional=True, batch_first=True, **kwargs):
        super(Net_block, self).__init__()
        self.lstm = nn.LSTM(input_dims, internal_dims, num_layers=1, bias=True, bidirectional=True, batch_first=True)
        if bidirectional:
            self.fc =nn.Linear(internal_dims*2, output_dims)
        else:
            self.fc =nn.Linear(internal_dims, output_dims)
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization/notebook
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name:
                if 'weight' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)
    
    def forward(self, x, hw=None):
        y, _ = self.lstm(x)
        y = self.fc(y) + x
        if hw is not None:
            y = y * hw
        return y
        
class Net_test(torch.nn.Module):
    def __init__(self, evaluate_dic_list, net_block_num=net_block_num, hw_active=hw_active, **kwargs):
        super(Net_test, self).__init__()
        self.hw_layer = hw_layer(evaluate_dic_list)
        self.hw_dims = self.hw_layer.channels
        
        self.net_block_list = nn.ModuleList()
        for _ in range(net_block_num):
            self.net_block_list.append(Net_block(self.hw_dims, self.hw_dims, self.hw_dims))
        
        self.fc = nn.Sequential(nn.Linear(self.hw_dims, 32, bias=False),
                                nn.SELU(),
                                nn.Linear(32, 1, bias=True))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization/notebook
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name:
                if 'weight' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)
    
    def forward(self, x):
        hw = self.hw_layer(x)
        x = hw
        
        for net_block in self.net_block_list:
            if hw_active:
                x = net_block(x, hw)
            else:
                x = net_block(x)
        
        x = self.fc(x)
        return x

In [ ]:
data_train_df = pd.read_csv('./Database/train.csv')
data_test_df = pd.read_csv('./Database/test.csv')

In [ ]:
drop_columns = ['pressure','id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2']
x_columns = [col for col in data_train_df.columns if col not in drop_columns]
y_columns = ['pressure']

In [ ]:
data_train = data_train_df[x_columns].values.astype(np.float32)
data_train = data_train.reshape(-1, 80, data_train.shape[-1])

target_train = data_train_df[y_columns].values.astype(np.float32)
target_train = target_train.reshape(-1, 80, target_train.shape[-1])

data_test = data_test_df[x_columns].values.astype(np.float32)
data_test = data_test.reshape(-1, 80, data_test.shape[-1])

In [ ]:
print(data_train.shape)

In [ ]:
np.random.seed(121212)
data_idx = np.arange(len(data_train))
np.random.shuffle(data_idx)

train_index = data_idx[:int(len(data_idx)*0.08)]
valid_index = data_idx[-int(len(data_idx)*0.02):]

print(len(train_index), len(valid_index))

x_train, y_train = data_train[train_index], target_train[train_index]
x_valid, y_valid = data_train[valid_index], target_train[valid_index]
x_test = data_test

In [ ]:
del data_train_df
del data_test_df
gc.collect()

In [ ]:
evaluate_list = [evaluate_build(x_test[..., i], evaluate_num) for i in range(x_test.shape[-1])]
evaluate_focus_list = []
for evaluate in evaluate_list:
    focus = 1 - (len(evaluate) - 1)/10
    if focus < focus_min:
        focus = focus_min
    evaluate_focus = focus_build(evaluate, focus)
    evaluate_focus_list.append(evaluate_focus)

In [ ]:
model = Net_test(evaluate_focus_list)
print(model)

train_parameter_num, freeze_parameter_num = net_parameter_count(model)
print(train_parameter_num, freeze_parameter_num)

In [ ]:
def train_submission(epoch_num, batch_size):
    dataLoads = dataLoads_build(x_train, y_train, x_valid, y_valid, batch_size)
    learn = Learner(dataLoads, model, loss_func=L1LossFlat())
    learn.lr_find()
    learn.fit_one_cycle(epoch_num, lr_max=2e-3, cbs=[ReduceLROnPlateau(monitor='valid_loss', min_delta=0.5, patience=10),
                                                     SaveModelCallback(monitor='valid_loss', fname=f'{fname}_B{batch_size}_best')])
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    state_dict = torch.load(f'models/{fname}_B{batch_size}_best.pth')
    model.load_state_dict(state_dict)

    loss = torch_valid([model.to(device)], L1LossFlat(), (x_train, y_train),  batch_size, to_device=device)
    valid_loss = torch_valid([model.to(device)], L1LossFlat(), (x_valid, y_valid),  batch_size, to_device=device)

    print(loss, valid_loss)

    predict = torch_predict([model.to(device)], x_test, batch_size, to_device=device)
    predict = np.reshape(predict, (-1))

    df = pd.read_csv('Database/sample_submission.csv', index_col=0)
    df['pressure'] = predict

    df.to_csv(f'Submission/{fname}_B{batch_size}.csv')

In [ ]:
train_submission(epoch_num_first, batch_size_first)

In [ ]:
0train_submission(epoch_num_second, batch_size_second)